## Introduction

### The notebook is a demo of [Encoding in Style: a StyleGAN Encoder for Image-to-Image Translation](https://arxiv.org/pdf/2008.00951.pdf) obtained from the authors' original [pixel2style2pixel implementation](https://github.com/eladrich/pixel2style2pixel).

<h3><center>Cartoonify results using Pixel2Style2Pixel Model</center></h3>
<img src="https://github.com/eladrich/pixel2style2pixel/raw/master/docs/toonify_input.jpg" width="900" height="750"/>
<img src="https://github.com/eladrich/pixel2style2pixel/raw/master/docs/toonify_output.jpg" width="900" height="750"/>
<h4></h4>
<h4><center><a href="https://github.com/eladrich/pixel2style2pixel">Source: Encoding in Style: a StyleGAN Encoder for Image-to-Image Translation [Elad Richardson et. al.]</a></center></h4>

## Acknowledgements

### This work was inspired by and derives codes from the official [Pixel2Style2Pixel implementation](https://github.com/eladrich/pixel2style2pixel). If you use this work, you should cite the research work [Encoding in Style: a StyleGAN Encoder for Image-to-Image Translation](https://arxiv.org/abs/2008.00951) and cite / star 🌟 the [official implementation](https://github.com/eladrich/pixel2style2pixel).

In [ ]:
import os
CODE_DIR = 'pixel2style2pixel'

!git clone https://github.com/eladrich/pixel2style2pixel.git $CODE_DIR
    
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!unzip ninja-linux.zip -d /usr/local/bin/
!update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 


os.makedirs('toonify_results')
os.chdir(f'./{CODE_DIR}')

In [ ]:
from argparse import Namespace
import time
import os
import cv2
import sys
import glob
import pprint
import random
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from datasets import augmentations
from utils.common import tensor2im, log_input_image
from models.psp import pSp

%load_ext autoreload
%autoreload 2

## Select Experiment Type
> Select which experiment you wish to perform inference on:
> 1. ffhq_encode
> 2. ffhq_frontalize
> 3. celebs_sketch_to_face
> 4. celebs_seg_to_face
> 5. celebs_super_resolution
> 6. toonify

In [ ]:
# experiment_type = 'ffhq_encode'
# experiment_type = 'ffhq_frontalize'
# experiment_type = 'celebs_sketch_to_face'
# experiment_type = 'celebs_seg_to_face'
# experiment_type = 'celebs_super_resolution'
experiment_type = 'toonify'

## Define Inference Parameters

> Below we have a dictionary defining parameters such as the path to the pretrained model to use and the path to the image to perform inference on. While we provide default values to run this script, feel free to change as needed.

In [ ]:
EXPERIMENT_DATA_ARGS = {
    "toonify": {
        "model_path": "../../input/pixel2style2pixel-pretrained-checkpoints-pytorch/psp_ffhq_toonify.pt",
        "image_path": "../../input/celeba-dataset/img_align_celeba/img_align_celeba/000020.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

In [ ]:
EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS[experiment_type]

## Load Pretrained Model
> We assume that you have downloaded all relevant models and placed them in the directory defined by the above dictionary.

In [ ]:
model_path = EXPERIMENT_ARGS['model_path']
ckpt = torch.load(model_path, map_location='cpu')

In [ ]:
opts = ckpt['opts']
pprint.pprint(opts)

In [ ]:
# update the training options
opts['checkpoint_path'] = model_path
if 'learn_in_w' not in opts:
    opts['learn_in_w'] = False

In [ ]:
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

## Visualize Input

In [ ]:
image_path = EXPERIMENT_DATA_ARGS[experiment_type]["image_path"]
original_image = Image.open(image_path)

In [ ]:
input_image = original_image.resize((256, 256))
input_image

## Perform Inference

In [ ]:
img_transforms = EXPERIMENT_ARGS['transform']
transformed_image = img_transforms(input_image)

In [ ]:
latent_mask = None

def run_on_batch(inputs, net, latent_mask=None):
    result_batch = []
    for image_idx, input_image in enumerate(inputs):
        # get latent vector to inject into our input image
        vec_to_inject = np.random.randn(1, 512).astype('float32')
        _, latent_to_inject = net(torch.from_numpy(vec_to_inject).to("cuda"),
                                  input_code=True,
                                  return_latents=True)
        # get output image with injected style vector
        res = net(input_image.unsqueeze(0).to("cuda").float(),
                  latent_mask=latent_mask,
                  inject_latent=latent_to_inject)
        result_batch.append(res)
    result_batch = torch.cat(result_batch, dim=0)
    return result_batch

In [ ]:
with torch.no_grad():
    tic = time.time()
    result_image = run_on_batch(transformed_image.unsqueeze(0), net, latent_mask)[0]
    toc = time.time()
    print('Inference took {:.4f} seconds.'.format(toc - tic))

### Visualize Result

In [ ]:
input_vis_image = log_input_image(transformed_image, opts)
output_image = tensor2im(result_image)

In [ ]:
res = np.concatenate([np.array(input_vis_image.resize((256, 256))),
                      np.array(output_image.resize((256, 256)))], axis=1)

In [ ]:
res_image = Image.fromarray(res)

res_image.save('../toonify_results/sample.jpg')
res_image

In [ ]:
res_image = Image.fromarray(res)
res_image

In [ ]:
image_paths = random.sample(glob.glob(os.path.join('../../input/celeba-dataset/img_align_celeba/img_align_celeba/', '*.jpg')), 100)

In [ ]:
image_paths[0].split('/')[-1]

In [ ]:
for image_path in image_paths:
    input_image = Image.open(image_path)
    input_image = input_image.resize((256, 256))
    img_transforms = EXPERIMENT_ARGS['transform']
    transformed_image = img_transforms(input_image)
    
    with torch.no_grad():
        result_image = run_on_batch(transformed_image.unsqueeze(0), net, latent_mask)[0]
        
    input_vis_image = log_input_image(transformed_image, opts)
    output_image = tensor2im(result_image)
    res = np.concatenate([np.array(input_vis_image.resize((256, 256))), np.array(output_image.resize((256, 256)))], axis=1)
    res_image = Image.fromarray(res)
    res_image.save(f"../toonify_results/{image_path.split('/')[-1]}")

In [ ]:
os.chdir(f'../')

In [ ]:
!rm -rf pixel2stylepixel
!rm ninja-linux.zip